<a href="https://colab.research.google.com/github/entc-17-fyp-20/CM_Normal_Behaviour_Modeling/blob/samal/CNN1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libries

In [771]:
# multivariate data preparation
from numpy import array
from numpy import hstack
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

import plotly.express as px
import plotly.graph_objects as go

# Select Turbine

In [517]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [772]:
#@title Select Dataset New { display-mode: "form" }
turbine = 'T01' #@param ["T01", "T02","T03","T22","T34","T54"]

path = '/content/drive/MyDrive/Merged/' + turbine + '.csv'
df = pd.read_csv(path)
df['Date_Time'] = pd.to_datetime(df['Date_Time'], format="%Y-%m-%d %H:%M:%S")
df = df.set_index(df['Date_Time'])

# Split in to dataframes

In [773]:
df1 = df[:'2017-03-19 05:10:00']
df2 = df['2017-04-13 16:10:00':"2017-05-31 00:00:00"]
df3 = df["2017-06-01 00:00:00":"2017-07-20 02:50:00"]
df4 = df['2017-07-21 08:10:00':'2017-08-14 05:20:00']
df5 = df['2018-01-15 12:00:00':"2018-02-10 23:50:00"]
df6 = df['2018-02-13 00:00:00':'2018-04-21 06:40:00']
df7 = df['2018-05-01 10:10:00':"2018-05-31 00:00:00"]
df8 = df['2018-06-01 00:00:00':'2018-08-14 06:50:00']
df9 = df['2018-08-21 07:00:00':]

# Time series plot

In [774]:
#@title Select feature to plot { display-mode: "form" }
column = "Wind_Speed" #@param ['Ambient_Temperature', 'Gear_Bearing_Temperature','Gear_Oil_Tempeature', 'Generator_Bearing_Tempeature', 'Generator_RPM','Possible_Active_Power', 'Active_Power', 'Reactive_Power', 'Voltage1','Voltage2', 'Voltage3', 'Hydrulic_Oil_Pressure','Hydrulic_Oil_Temperature', 'Wind_Speed'] 


fig = px.line(df1, x="Date_Time" , y=column) 
fig.show()

# Select the dataframe and features to CNN

In [775]:
cols = df1.columns.tolist()
cols=cols[1:6]+[cols[-1]]+[cols[7]];
df1=df1[cols];

# train, test dataset split

In [776]:
X=df1[cols[:-1]];
y=df1[cols[-1]];

X=np.array(X);
X.shape

(11120, 6)

# Build CNN model

In [777]:
from sklearn.datasets import load_boston
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [778]:
X = X.reshape(X.shape[0], X.shape[1], 1)

In [779]:
xtrain, xtest, ytrain, ytest=train_test_split(X, y, test_size=0.15) 

In [780]:
model = Sequential()
model.add(Conv1D(64, 4, activation="relu", input_shape=(6, 1)))
model.add(Conv1D(32, 2, activation="relu", input_shape=(3, 34)))
model.add(Flatten())
model.add(Dense(32, activation="relu"))
model.add(Dense(1))
model.compile(loss="mse", optimizer="adam")

model.summary()

Model: "sequential_52"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_68 (Conv1D)           (None, 3, 64)             320       
_________________________________________________________________
conv1d_69 (Conv1D)           (None, 2, 32)             4128      
_________________________________________________________________
flatten_46 (Flatten)         (None, 64)                0         
_________________________________________________________________
dense_90 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_91 (Dense)             (None, 1)                 33        
Total params: 6,561
Trainable params: 6,561
Non-trainable params: 0
_________________________________________________________________


# Train and test the model

In [781]:
model.fit(xtrain, ytrain, batch_size=10,epochs=300, verbose=0)
ypred = model.predict(xtest)
ypred = ypred.reshape(ypred.shape[0],)
error = abs(ypred-ytest)
print(abs(error).mean())

72.6808998426945


In [782]:
ytest = pd. DataFrame(ytest) 

In [783]:
ypred = pd. DataFrame({'Active_Power_pred': ypred})

In [784]:
error = pd. DataFrame({'error': error}) 
fig = px.line(error, y='error') 
fig.show()

In [785]:
ytest.reset_index(drop=True, inplace=True)

In [786]:
df_pred_test = pd.concat([ypred, ytest], axis=1)

In [787]:
df_long=pd.melt(df_pred_test, value_vars=['Active_Power_pred','Active_Power'])

# plotly 
fig = px.line(df_long,y='value', color='variable')

# Show plot 
fig.show()

# Test on Different dataframe

In [788]:
df5 = df['2018-01-15 12:00:00':"2018-02-10 23:50:00"]
cols = df5.columns.tolist()
cols=cols[1:6]+[cols[-1]]+[cols[7]];
df5=df5[cols];

X=df5[cols[:-1]];
y=df5[cols[-1]];

X=np.array(X);
X = X.reshape(X.shape[0], X.shape[1], 1)

ypred = model.predict(X)

ypred = ypred.reshape(ypred.shape[0],)
error = abs(ypred-y)
ypred = pd. DataFrame({'Active_Power_pred': ypred})
y.reset_index(drop=True, inplace=True)
df_pred_test_1 = pd.concat([ypred, y], axis=1)
df_long=pd.melt(df_pred_test_1, value_vars=['Active_Power_pred','Active_Power'])

# plotly 
fig = px.line(df_long,y='value', color='variable')

# Show plot 
fig.show()

In [789]:
error = pd. DataFrame({'error': error}) 
fig = px.line(error, y='error') 
fig.show()

In [790]:
df7 = df['2018-05-01 10:10:00':"2018-05-31 00:00:00"]
cols = df7.columns.tolist()
cols=cols[1:6]+[cols[-1]]+[cols[7]];
df7=df7[cols];

X=df7[cols[:-1]];
y=df7[cols[-1]];

X=np.array(X);
X.shape

X = X.reshape(X.shape[0], X.shape[1], 1)

ypred = model.predict(X)
ypred = ypred.reshape(ypred.shape[0],)
error = abs(ypred-y)

ypred = pd. DataFrame({'Active_Power_pred': ypred})
y.reset_index(drop=True, inplace=True)
df_pred_test_2 = pd.concat([ypred, y], axis=1)
df_long=pd.melt(df_pred_test_2, value_vars=['Active_Power_pred','Active_Power'])

# plotly 
fig = px.line(df_long,y='value', color='variable')

# Show plot 
fig.show()

In [791]:
error = pd. DataFrame({'error': error}) 
fig = px.line(error, y='error') 
fig.show()

# T34 - Dec27 Failure 

In [809]:
#@title Select Dataset New { display-mode: "form" }
turbine = 'T34' #@param ["T01", "T02","T03","T22","T34","T54"]

path = '/content/drive/MyDrive/Merged/' + turbine + '.csv'
df = pd.read_csv(path)
df['Date_Time'] = pd.to_datetime(df['Date_Time'], format="%Y-%m-%d %H:%M:%S")
df = df.set_index(df['Date_Time'])
df=df['2018-12-26':'2018-12-31']
#df=df['2018-06-10':'2018-08-07']

In [810]:
cols = df.columns.tolist()
cols=cols[1:6]+[cols[-1]]+[cols[7]];
df=df[cols];

X=df[cols[:-1]];
y=df[cols[-1]];

X=np.array(X);
X.shape

X = X.reshape(X.shape[0], X.shape[1], 1)

ypred = model.predict(X)
ypred = ypred.reshape(ypred.shape[0],)
error = abs(ypred-y)

ypred = pd. DataFrame({'Active_Power_pred': ypred})
y.reset_index(drop=True, inplace=True)
df_pred_test_2 = pd.concat([ypred, y], axis=1)
df_long=pd.melt(df_pred_test_2, value_vars=['Active_Power_pred','Active_Power'])

# plotly 
fig = px.line(df_long,y='value', color='variable')

# Show plot 
fig.show()

In [811]:
df = pd.read_csv(path)
df['Date_Time'] = pd.to_datetime(df['Date_Time'], format="%Y-%m-%d %H:%M:%S")
df = df.set_index(df['Date_Time'])
df=df['2018-12-26':'2018-12-31']
df['error']=error
fig = px.line(df, x="Date_Time" , y='error') 
fig.show()